In [1]:
import joblib
from flask import Flask, request, json, jsonify, render_template
from werkzeug.exceptions import HTTPException

In [2]:
print('Loading model...')
MODEL_PATH = 'models/model.joblib'
print('Done!')

Loading model...
Done!


In [3]:
print('Loading app...')
app = Flask(__name__)
print('Done!')

Loading app...
Done!


In [4]:
@app.errorhandler(HTTPException)
def handle_exception(e):
    """Return JSON instead of HTML for HTTP errors (which is the basic error
    response with Flask).
    """
    # Start with the correct headers and status code from the error
    response = e.get_response()
    # Replace the body with JSON
    response.data = json.dumps({
        "code": e.code,
        "name": e.name,
        "description": e.description,
    })
    response.content_type = "application/json"
    return response

In [5]:
class MissingKeyError(HTTPException):
    # We can define our own error for the missing key
    code = 422
    name = "Missing key error"
    description = "JSON content missing key 'input'."

In [6]:
class MissingJSON(HTTPException):
    # We can define our own error for missing JSON
    code = 400
    name = "Missing JSON"
    description = "Missing JSON."

In [10]:
input_simple = {"input": [[7.0, 0.27, 0.36, 20.7, 0.045, 45.0, 170.0, 1.001, 3.0, 0.45, 8.8]]}

In [27]:
type(input_simple)

dict

In [28]:
#Return a prediction with our classification model.
def make_prediction(input_simple: float):
    # Load model
    classifier = joblib.load(MODEL_PATH)
    # Make prediction (the model expects a 2D array that is why we put input in a list of list) and return it
    prediction = classifier.predict([[input_simple]])
    return prediction[0]

In [32]:
make_prediction(input_simple)

C:\Users\javie\anaconda3\lib\site-packages\sklearn\base.py:333: UserWarning: Trying to unpickle estimator SimpleImputer from version 1.0.1 when using version 1.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  UserWarning,
C:\Users\javie\anaconda3\lib\site-packages\sklearn\base.py:333: UserWarning: Trying to unpickle estimator StandardScaler from version 1.0.1 when using version 1.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  UserWarning,
C:\Users\javie\anaconda3\lib\site-packages\sklearn\base.py:333: UserWarning: Trying to unpickle estimator DecisionTreeClassifier from version 1.0.1 when using version 1.0. This might lead to breaking code or invalid results. Use at

TypeError: float() argument must be a string or a number, not 'dict'

In [ ]:
classifier = joblib.load(MODEL_PATH)

In [30]:
prediction = classifier.predict([[input_simple]])

C:\Users\javie\anaconda3\lib\site-packages\sklearn\base.py:442: UserWarning: X does not have valid feature names, but SimpleImputer was fitted with feature names
  "X does not have valid feature names, but"


TypeError: float() argument must be a string or a number, not 'dict'

In [13]:
print('Predicting...')
@app.route("/predict", methods=["POST"])
def predict():
    # Check parameters
    if request.json:
        # Get JSON as dictionnary
        json_input = request.get_json()
       
        if "input" not in json_input:
            # If 'input' is not in our JSON we raise our own error
            raise MissingKeyError()
        # Call our predict function that handle loading model and making a
        # prediction
        prediction = make_prediction(json_input["input"])
        # Return prediction
        response = {
            # Since prediction is a float and jsonify function can't handle
            # floats we need to convert it to string
            "quality": str(prediction),
        }
        return jsonify(response), 200
    raise MissingJSON()
print('Done!')

Predicting...
Done!


In [14]:
@app.route("/")
def index():
    return render_template("index.html")

In [16]:
if __name__ == "__main__":
    app.run(debug=True)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


 * Restarting with windowsapi reloader


SystemExit: 1